In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 1) 데이터 준비: MNIST 불러오기 + 숫자 이미지를 텐서로 바꾸고(0~1) 정규화
transform = transforms.Compose([
    transforms.ToTensor(),                   # (H,W) 이미지를 (C,H,W) 텐서로 + 0~1 스케일
    transforms.Normalize((0.1307,), (0.3081,))  # MNIST 평균/표준편차로 정규화(학습 안정화)
])

train_ds = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_ds  = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_ds, batch_size=256, shuffle=False)



100%|██████████| 9.91M/9.91M [00:00<00:00, 18.0MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 483kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.46MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 7.68MB/s]


In [2]:
# 2) CNN 모델 만들기
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        # 특징 찾기(돋보기): 1채널(흑백) -> 32채널(특징 32종)
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        # 특징 더 찾기: 32 -> 64
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)

        # 크기 줄이기(핵심만): 2x2에서 최대값
        self.pool = nn.MaxPool2d(2, 2)

        # 과적합 방지(선택): 일부를 랜덤으로 끄기
        self.drop = nn.Dropout(0.25)

        # 최종 판단기(분류기)
        # 입력 이미지가 28x28
        # conv1(패딩1이라 크기 유지) -> 28x28, pool -> 14x14
        # conv2 -> 14x14, pool -> 7x7
        # 채널 64개니까: 64*7*7 개를 한 줄로 펼쳐서 10개(0~9)로
        self.fc = nn.Linear(64 * 7 * 7, 10)

    def forward(self, x):
        # 1) 특징 찾기 + 활성화(ReLU)
        x = torch.relu(self.conv1(x))
        # 2) 크기 줄이기
        x = self.pool(x)

        # 3) 더 깊은 특징 찾기 + 활성화
        x = torch.relu(self.conv2(x))
        # 4) 다시 크기 줄이기
        x = self.pool(x)

        # 5) 펼치기(덩어리 -> 한 줄)
        x = x.view(x.size(0), -1)

        # 6) 드롭아웃(학습 때만 일부 끄기)
        x = self.drop(x)

        # 7) 최종 점수표(10개) 출력
        x = self.fc(x)
        return x



In [3]:
# 3) 학습 준비: 기기(CPU/GPU), 손실함수, 옵티마이저
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SimpleCNN().to(device)

criterion = nn.CrossEntropyLoss()                 # “정답과 얼마나 다른지” 채점
optimizer = optim.Adam(model.parameters(), lr=1e-3)  # “어떻게 고칠지” 선생님(Adam)


In [4]:
# 4) 학습 루프
def train_one_epoch():
    model.train()  # 지금부터 학습 모드(드롭아웃 ON 등)
    total_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()          # (중요) 이전 미분값(기울기) 지우기
        logits = model(images)         # 모델 통과 -> 점수표(softmax 전)
        loss = criterion(logits, labels)  # 채점(손실)
        loss.backward()                # “어디를 얼마나 고쳐야 하는지” 계산
        optimizer.step()               # 실제로 가중치 업데이트(공부)

        total_loss += loss.item()

    return total_loss / len(train_loader)


In [5]:
# 5) 평가(테스트 정확도)
@torch.no_grad()
def evaluate():
    model.eval()  # 평가 모드(드롭아웃 OFF 등)
    correct = 0
    total = 0

    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        logits = model(images)
        preds = logits.argmax(dim=1)   # 점수표에서 제일 큰 값이 예측
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    return correct / total


In [6]:

# 6) 실제 실행
epochs = 3
for epoch in range(1, epochs + 1):
    loss = train_one_epoch()
    acc = evaluate()
    print(f"Epoch {epoch} | loss: {loss:.4f} | test acc: {acc*100:.2f}%")


Epoch 1 | loss: 0.1467 | test acc: 98.19%
Epoch 2 | loss: 0.0540 | test acc: 98.57%
Epoch 3 | loss: 0.0396 | test acc: 99.00%
